In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
parasharmanas_movie_recommendation_system_path = kagglehub.dataset_download('parasharmanas/movie-recommendation-system')

print('Data source import complete.')


100%|██████████| 165M/165M [00:00<00:00, 246MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install scikit-surprise tensorflow wurlitzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=173954cf1ee58fd600049878479fae3c8fd0b80ae6d2d3576703cc3f4828fa6f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import scipy.sparse
import tensorflow as tf

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split as sk_train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import normalize
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from collections import defaultdict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten, Conv1D, MaxPooling1D, ZeroPadding1D
from math import sqrt
import pickle
from google.colab import files
import zipfile
import os
from keras.models import load_model
import joblib

In [ ]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d parasharmanas/movie-recommendation-system

# Crie um diretório onde os arquivos extraídos serão armazenados
if not os.path.exists("movie-recommendation-system"):
    os.makedirs("movie-recommendation-system")

with zipfile.ZipFile("movie-recommendation-system.zip", "r") as zip_ref:
    zip_ref.extractall("movie-recommendation-system")


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system
License(s): ODbL-1.0
 98% 162M/165M [00:00<00:00, 303MB/s]
100% 165M/165M [00:00<00:00, 278MB/s]


In [ ]:
movies_file = 'movie-recommendation-system/movies.csv'
movies = pd.read_csv(movies_file)
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
users_file = 'movie-recommendation-system/ratings.csv'
users = pd.read_csv(users_file)
users.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [ ]:
# Adicionando Amanda a base
new_data = [
  {'userId': 162544, 'movieId': 195159, 'rating': 5.0, 'timestamp': '1618387220'},
  {'userId': 162544, 'movieId': 7099, 'rating': 3.0, 'timestamp': '1618387221'},
  {'userId': 162544, 'movieId': 6350, 'rating': 4.0, 'timestamp': '1618387222'},
  {'userId': 162544, 'movieId': 1223, 'rating': 2.0, 'timestamp': '1618387223'},
  {'userId': 162544, 'movieId': 60069, 'rating': 5.0, 'timestamp': '1618387224'},
  {'userId': 162544, 'movieId': 157777, 'rating': 0.0, 'timestamp': '1618387225'},
  {'userId': 162544, 'movieId': 208767, 'rating': 0.0, 'timestamp': '1618387226'},
  {'userId': 162544, 'movieId': 1274, 'rating': 0.0, 'timestamp': '1618387227'},
  {'userId': 162544, 'movieId': 140295, 'rating': 0.0, 'timestamp': '1618387228'},
  {'userId': 162544, 'movieId': 2761, 'rating': 2.0, 'timestamp': '1618387229'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000095,162544,195159,5.0,1618387220
25000096,162544,7099,3.0,1618387221
25000097,162544,6350,4.0,1618387222
25000098,162544,1223,2.0,1618387223
25000099,162544,60069,5.0,1618387224
25000100,162544,157777,0.0,1618387225
25000101,162544,208767,0.0,1618387226
25000102,162544,1274,0.0,1618387227
25000103,162544,140295,0.0,1618387228
25000104,162544,2761,2.0,1618387229


In [ ]:
# Adicionando Yasmin a base
new_data = [
  {'userId': 162545, 'movieId': 135585, 'rating': 4.0, 'timestamp': '1618387230'},
  {'userId': 162545, 'movieId': 172281, 'rating': 2.0, 'timestamp': '1618387231'},
  {'userId': 162545, 'movieId': 144494, 'rating': 3.0, 'timestamp': '1618387232'},
  {'userId': 162545, 'movieId': 149830, 'rating': 4.0, 'timestamp': '1618387233'},
  {'userId': 162545, 'movieId': 27781, 'rating': 3.0, 'timestamp': '1618387234'},
  {'userId': 162545, 'movieId': 27658, 'rating': 2.0, 'timestamp': '1618387235'},
  {'userId': 162545, 'movieId': 2793, 'rating': 3.0, 'timestamp': '1618387236'},
  {'userId': 162545, 'movieId': 136946, 'rating': 2.0, 'timestamp': '1618387237'},
  {'userId': 162545, 'movieId': 151367, 'rating': 3.0, 'timestamp': '1618387238'},
  {'userId': 162545, 'movieId': 169746, 'rating': 3.0, 'timestamp': '1618387239'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000105,162545,135585,4.0,1618387230
25000106,162545,172281,2.0,1618387231
25000107,162545,144494,3.0,1618387232
25000108,162545,149830,4.0,1618387233
25000109,162545,27781,3.0,1618387234
25000110,162545,27658,2.0,1618387235
25000111,162545,2793,3.0,1618387236
25000112,162545,136946,2.0,1618387237
25000113,162545,151367,3.0,1618387238
25000114,162545,169746,3.0,1618387239


In [ ]:
# Adicionando Mauricio a base
new_data = [
  {'userId': 162546, 'movieId': 8690, 'rating': 4.0, 'timestamp': '1618387240'},
  {'userId': 162546, 'movieId': 173707, 'rating': 3.0, 'timestamp': '1618387241'},
  {'userId': 162546, 'movieId': 171811, 'rating': 3.0, 'timestamp': '1618387242'},
  {'userId': 162546, 'movieId': 97724, 'rating': 2.0, 'timestamp': '1618387243'},
  {'userId': 162546, 'movieId': 116819, 'rating': 1.0, 'timestamp': '1618387244'},
  {'userId': 162546, 'movieId': 79681, 'rating': 2.0, 'timestamp': '1618387245'},
  {'userId': 162546, 'movieId': 138280, 'rating': 0.0, 'timestamp': '1618387246'},
  {'userId': 162546, 'movieId': 156305, 'rating': 2.0, 'timestamp': '1618387247'},
  {'userId': 162546, 'movieId': 169746, 'rating': 0.0, 'timestamp': '1618387248'},
  {'userId': 162546, 'movieId': 171973, 'rating': 1.0, 'timestamp': '1618387249'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000115,162546,8690,4.0,1618387240
25000116,162546,173707,3.0,1618387241
25000117,162546,171811,3.0,1618387242
25000118,162546,97724,2.0,1618387243
25000119,162546,116819,1.0,1618387244
25000120,162546,79681,2.0,1618387245
25000121,162546,138280,0.0,1618387246
25000122,162546,156305,2.0,1618387247
25000123,162546,169746,0.0,1618387248
25000124,162546,171973,1.0,1618387249


In [ ]:
# Adicionando Neylon a base
new_data = [
  {'userId': 162547, 'movieId': 128985, 'rating': 5.0, 'timestamp': '1618387250'},
  {'userId': 162547, 'movieId': 5018, 'rating': 1.0, 'timestamp': '1618387251'},
  {'userId': 162547, 'movieId': 175047, 'rating': 3.0, 'timestamp': '1618387252'},
  {'userId': 162547, 'movieId': 187483, 'rating': 1.0, 'timestamp': '1618387253'},
  {'userId': 162547, 'movieId': 201869, 'rating': 3.0, 'timestamp': '1618387254'},
  {'userId': 162547, 'movieId': 6016, 'rating': 5.0, 'timestamp': '1618387255'},
  {'userId': 162547, 'movieId': 87260, 'rating': 0.0, 'timestamp': '1618387256'},
  {'userId': 162547, 'movieId': 122339, 'rating': 1.0, 'timestamp': '1618387257'},
  {'userId': 162547, 'movieId': 6902, 'rating': 4.0, 'timestamp': '1618387258'},
  {'userId': 162547, 'movieId': 49530, 'rating': 3.0, 'timestamp': '1618387259'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000125,162547,128985,5.0,1618387250
25000126,162547,5018,1.0,1618387251
25000127,162547,175047,3.0,1618387252
25000128,162547,187483,1.0,1618387253
25000129,162547,201869,3.0,1618387254
25000130,162547,6016,5.0,1618387255
25000131,162547,87260,0.0,1618387256
25000132,162547,122339,1.0,1618387257
25000133,162547,6902,4.0,1618387258
25000134,162547,49530,3.0,1618387259


In [ ]:
# Adicionando Jeniffer a base
new_data = [
  {'userId': 162548, 'movieId': 26236, 'rating': 1.0, 'timestamp': '1618387260'},
  {'userId': 162548, 'movieId': 70223, 'rating': 2.0, 'timestamp': '1618387261'},
  {'userId': 162548, 'movieId': 4802, 'rating': 1.0, 'timestamp': '1618387262'},
  {'userId': 162548, 'movieId': 5541, 'rating': 0.0, 'timestamp': '1618387263'},
  {'userId': 162548, 'movieId': 115862, 'rating': 3.0, 'timestamp': '1618387264'},
  {'userId': 162548, 'movieId': 68831, 'rating': 1.0, 'timestamp': '1618387265'},
  {'userId': 162548, 'movieId': 87745, 'rating': 1.0, 'timestamp': '1618387266'},
  {'userId': 162548, 'movieId': 137176, 'rating': 3.0, 'timestamp': '1618387267'},
  {'userId': 162548, 'movieId': 209155, 'rating': 0.0, 'timestamp': '1618387268'},
  {'userId': 162548, 'movieId': 133959, 'rating': 1.0, 'timestamp': '1618387269'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000135,162548,26236,1.0,1618387260
25000136,162548,70223,2.0,1618387261
25000137,162548,4802,1.0,1618387262
25000138,162548,5541,0.0,1618387263
25000139,162548,115862,3.0,1618387264
25000140,162548,68831,1.0,1618387265
25000141,162548,87745,1.0,1618387266
25000142,162548,137176,3.0,1618387267
25000143,162548,209155,0.0,1618387268
25000144,162548,133959,1.0,1618387269


In [ ]:
# Adicionando Sue a base
new_data = [
  {'userId': 162549, 'movieId': 164851, 'rating': 3.0, 'timestamp': '1618387270'},
  {'userId': 162549, 'movieId': 5971, 'rating': 4.0, 'timestamp': '1618387271'},
  {'userId': 162549, 'movieId': 168918, 'rating': 5.0, 'timestamp': '1618387272'},
  {'userId': 162549, 'movieId': 26662, 'rating': 1.0, 'timestamp': '1618387273'},
  {'userId': 162549, 'movieId': 141636, 'rating': 2.0, 'timestamp': '1618387274'},
  {'userId': 162549, 'movieId': 175189, 'rating': 2.0, 'timestamp': '1618387275'},
  {'userId': 162549, 'movieId': 68954, 'rating': 5.0, 'timestamp': '1618387276'},
  {'userId': 162549, 'movieId': 134853, 'rating': 3.0, 'timestamp': '1618387277'},
  {'userId': 162549, 'movieId': 138702, 'rating': 5.0, 'timestamp': '1618387278'},
  {'userId': 162549, 'movieId': 61236, 'rating': 4.0, 'timestamp': '1618387279'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000145,162549,164851,3.0,1618387270
25000146,162549,5971,4.0,1618387271
25000147,162549,168918,5.0,1618387272
25000148,162549,26662,1.0,1618387273
25000149,162549,141636,2.0,1618387274
25000150,162549,175189,2.0,1618387275
25000151,162549,68954,5.0,1618387276
25000152,162549,134853,3.0,1618387277
25000153,162549,138702,5.0,1618387278
25000154,162549,61236,4.0,1618387279


In [ ]:
# Adicionando Francine a base
new_data = [
  {'userId': 162550, 'movieId': 1464, 'rating': 3.0, 'timestamp': '1618387280'},
  {'userId': 162550, 'movieId': 62304, 'rating': 4.0, 'timestamp': '1618387281'},
  {'userId': 162550, 'movieId': 121051, 'rating': 0.0, 'timestamp': '1618387282'},
  {'userId': 162550, 'movieId': 139703, 'rating': 2.0, 'timestamp': '1618387283'},
  {'userId': 162550, 'movieId': 149454, 'rating': 0.0, 'timestamp': '1618387284'},
  {'userId': 162550, 'movieId': 151569, 'rating': 5.0, 'timestamp': '1618387285'},
  {'userId': 162550, 'movieId': 187483, 'rating': 1.0, 'timestamp': '1618387286'},
  {'userId': 162550, 'movieId': 193129, 'rating': 4.0, 'timestamp': '1618387287'},
  {'userId': 162550, 'movieId': 196359, 'rating': 0.0, 'timestamp': '1618387288'},
  {'userId': 162550, 'movieId': 200286, 'rating': 5.0, 'timestamp': '1618387289'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000155,162550,1464,3.0,1618387280
25000156,162550,62304,4.0,1618387281
25000157,162550,121051,0.0,1618387282
25000158,162550,139703,2.0,1618387283
25000159,162550,149454,0.0,1618387284
25000160,162550,151569,5.0,1618387285
25000161,162550,187483,1.0,1618387286
25000162,162550,193129,4.0,1618387287
25000163,162550,196359,0.0,1618387288
25000164,162550,200286,5.0,1618387289


In [ ]:
# Adicionando Thiago a base
new_data = [
  {'userId': 162551, 'movieId': 4719, 'rating': 4.0, 'timestamp': '1618387300'},
  {'userId': 162551, 'movieId': 172197, 'rating': 3.0, 'timestamp': '1618387301'},
  {'userId': 162551, 'movieId': 26184, 'rating': 3.0, 'timestamp': '1618387302'},
  {'userId': 162551, 'movieId': 8253, 'rating': 0.0, 'timestamp': '1618387303'},
  {'userId': 162551, 'movieId': 27328, 'rating': 5.0, 'timestamp': '1618387304'},
  {'userId': 162551, 'movieId': 3104, 'rating': 4.0, 'timestamp': '1618387305'},
  {'userId': 162551, 'movieId': 151781, 'rating': 1.0, 'timestamp': '1618387306'},
  {'userId': 162551, 'movieId': 96775, 'rating': 4.0, 'timestamp': '1618387307'},
  {'userId': 162551, 'movieId': 91542, 'rating': 5.0, 'timestamp': '1618387308'},
  {'userId': 162551, 'movieId': 69136, 'rating': 2.0, 'timestamp': '1618387309'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000165,162551,4719,4.0,1618387300
25000166,162551,172197,3.0,1618387301
25000167,162551,26184,3.0,1618387302
25000168,162551,8253,0.0,1618387303
25000169,162551,27328,5.0,1618387304
25000170,162551,3104,4.0,1618387305
25000171,162551,151781,1.0,1618387306
25000172,162551,96775,4.0,1618387307
25000173,162551,91542,5.0,1618387308
25000174,162551,69136,2.0,1618387309


In [ ]:
# Adicionando Marcos a base
new_data = [
  {'userId': 162552, 'movieId': 140511, 'rating': 5.0, 'timestamp': '1618387400'},
  {'userId': 162552, 'movieId': 136451, 'rating': 5.0, 'timestamp': '1618387401'},
  {'userId': 162552, 'movieId': 196965, 'rating': 3.0, 'timestamp': '1618387402'},
  {'userId': 162552, 'movieId': 199358, 'rating': 4.0, 'timestamp': '1618387403'},
  {'userId': 162552, 'movieId': 3000, 'rating': 5.0, 'timestamp': '1618387404'},
  {'userId': 162552, 'movieId': 6350, 'rating': 5.0, 'timestamp': '1618387405'},
  {'userId': 162552, 'movieId': 174703, 'rating': 1.0, 'timestamp': '1618387406'},
  {'userId': 162552, 'movieId': 157777, 'rating': 4.0, 'timestamp': '1618387407'},
  {'userId': 162552, 'movieId': 193581, 'rating': 3.0, 'timestamp': '1618387408'},
  {'userId': 162552, 'movieId': 208293, 'rating': 1.0, 'timestamp': '1618387409'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000175,162552,140511,5.0,1618387400
25000176,162552,136451,5.0,1618387401
25000177,162552,196965,3.0,1618387402
25000178,162552,199358,4.0,1618387403
25000179,162552,3000,5.0,1618387404
25000180,162552,6350,5.0,1618387405
25000181,162552,174703,1.0,1618387406
25000182,162552,157777,4.0,1618387407
25000183,162552,193581,3.0,1618387408
25000184,162552,208293,1.0,1618387409


In [ ]:
# Adicionando Margarida a base
new_data = [
  {'userId': 162553, 'movieId': 194979, 'rating': 3.0, 'timestamp': '1618387500'},
  {'userId': 162553, 'movieId': 187043, 'rating': 3.0, 'timestamp': '1618387501'},
  {'userId': 162553, 'movieId': 6120, 'rating': 3.0, 'timestamp': '1618387502'},
  {'userId': 162553, 'movieId': 43932, 'rating': 2.0, 'timestamp': '1618387503'},
  {'userId': 162553, 'movieId': 167282, 'rating': 3.0, 'timestamp': '1618387504'},
  {'userId': 162553, 'movieId': 188983, 'rating': 4.0, 'timestamp': '1618387505'},
  {'userId': 162553, 'movieId': 201869, 'rating': 3.0, 'timestamp': '1618387506'},
  {'userId': 162553, 'movieId': 117734, 'rating': 2.0, 'timestamp': '1618387507'},
  {'userId': 162553, 'movieId': 6902, 'rating': 3.0, 'timestamp': '1618387508'},
  {'userId': 162553, 'movieId': 124781, 'rating': 2.0, 'timestamp': '1618387509'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000185,162553,194979,3.0,1618387500
25000186,162553,187043,3.0,1618387501
25000187,162553,6120,3.0,1618387502
25000188,162553,43932,2.0,1618387503
25000189,162553,167282,3.0,1618387504
25000190,162553,188983,4.0,1618387505
25000191,162553,201869,3.0,1618387506
25000192,162553,117734,2.0,1618387507
25000193,162553,6902,3.0,1618387508
25000194,162553,124781,2.0,1618387509


In [ ]:
# Adicionando Júlia a base
new_data = [
  {'userId': 162554, 'movieId': 169858, 'rating': 3.0, 'timestamp': '1618387600'},
  {'userId': 162554, 'movieId': 108316, 'rating': 4.0, 'timestamp': '1618387601'},
  {'userId': 162554, 'movieId': 123184, 'rating': 3.0, 'timestamp': '1618387602'},
  {'userId': 162554, 'movieId': 197075, 'rating': 1.0, 'timestamp': '1618387603'},
  {'userId': 162554, 'movieId': 152081, 'rating': 5.0, 'timestamp': '1618387604'},
  {'userId': 162554, 'movieId': 8961, 'rating': 5.0, 'timestamp': '1618387605'},
  {'userId': 162554, 'movieId': 194218, 'rating': 1.0, 'timestamp': '1618387606'},
  {'userId': 162554, 'movieId': 26340, 'rating': 3.0, 'timestamp': '1618387607'},
  {'userId': 162554, 'movieId': 1261, 'rating': 2.0, 'timestamp': '1618387608'},
  {'userId': 162554, 'movieId': 189203, 'rating': 3.0, 'timestamp': '1618387609'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000195,162554,169858,3.0,1618387600
25000196,162554,108316,4.0,1618387601
25000197,162554,123184,3.0,1618387602
25000198,162554,197075,1.0,1618387603
25000199,162554,152081,5.0,1618387604
25000200,162554,8961,5.0,1618387605
25000201,162554,194218,1.0,1618387606
25000202,162554,26340,3.0,1618387607
25000203,162554,1261,2.0,1618387608
25000204,162554,189203,3.0,1618387609


In [ ]:
# Adicionando Isaac a base
new_data = [
  {'userId': 162543, 'movieId': 159403, 'rating': 3.0, 'timestamp': '1618387200'},
  {'userId': 162543, 'movieId': 7099, 'rating': 4.0, 'timestamp': '1618387201'},
  {'userId': 162543, 'movieId': 6350, 'rating': 3.0, 'timestamp': '1618387202'},
  {'userId': 162543, 'movieId': 157777, 'rating': 2.0, 'timestamp': '1618387203'},
  {'userId': 162543, 'movieId': 47404, 'rating': 2.0, 'timestamp': '1618387204'},
  {'userId': 162543, 'movieId': 116905, 'rating': 5.0, 'timestamp': '1618387205'},
  {'userId': 162543, 'movieId': 27800, 'rating': 4.0, 'timestamp': '1618387206'},
  {'userId': 162543, 'movieId': 130508, 'rating': 5.0, 'timestamp': '1618387207'},
  {'userId': 162543, 'movieId': 5146, 'rating': 5.0, 'timestamp': '1618387208'},
  {'userId': 162543, 'movieId': 65796, 'rating': 3.0, 'timestamp': '1618387209'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000205,162543,159403,3.0,1618387200
25000206,162543,7099,4.0,1618387201
25000207,162543,6350,3.0,1618387202
25000208,162543,157777,2.0,1618387203
25000209,162543,47404,2.0,1618387204
25000210,162543,116905,5.0,1618387205
25000211,162543,27800,4.0,1618387206
25000212,162543,130508,5.0,1618387207
25000213,162543,5146,5.0,1618387208
25000214,162543,65796,3.0,1618387209


In [ ]:
# Adicionando Lais a base
new_data = [
  {'userId': 162542, 'movieId': 141367, 'rating': 0.0, 'timestamp': '1618387210'},
  {'userId': 162542, 'movieId': 151781, 'rating': 0.0, 'timestamp': '1618387211'},
  {'userId': 162542, 'movieId': 52281, 'rating': 0.0, 'timestamp': '1618387212'},
  {'userId': 162542, 'movieId': 93750, 'rating': 0.0, 'timestamp': '1618387213'},
  {'userId': 162542, 'movieId': 93748, 'rating': 0.0, 'timestamp': '1618387214'},
  {'userId': 162542, 'movieId': 140511, 'rating': 5.0, 'timestamp': '1618387215'},
  {'userId': 162542, 'movieId': 95773, 'rating': 0.0, 'timestamp': '1618387216'},
  {'userId': 162542, 'movieId': 6741, 'rating': 0.0, 'timestamp': '1618387217'},
  {'userId': 162542, 'movieId': 194750, 'rating': 0.0, 'timestamp': '1618387218'},
  {'userId': 162542, 'movieId': 206785, 'rating': 0.0, 'timestamp': '1618387219'}
]
new_data_df = pd.DataFrame(new_data)
users = pd.concat([users, new_data_df], ignore_index=True)
users.tail(10)

,userId,movieId,rating,timestamp
25000215,162542,141367,0.0,1618387210
25000216,162542,151781,0.0,1618387211
25000217,162542,52281,0.0,1618387212
25000218,162542,93750,0.0,1618387213
25000219,162542,93748,0.0,1618387214
25000220,162542,140511,5.0,1618387215
25000221,162542,95773,0.0,1618387216
25000222,162542,6741,0.0,1618387217
25000223,162542,194750,0.0,1618387218
25000224,162542,206785,0.0,1618387219


In [ ]:
movie_dataframe = pd.merge(users, movies, on='movieId')

movie_dataframe.head(10)
movie_dataframe.tail(30)

,userId,movieId,rating,timestamp,title,genres
25000195,162554,169858,3.0,1618387600,Kings of Kallstadt (2014),Children|Comedy|Documentary
25000196,162554,108316,4.0,1618387601,American Scary (2006),Comedy|Documentary|Horror
25000197,162554,123184,3.0,1618387602,The Secret Agent Club (1996),Action|Children|Comedy
25000198,162554,197075,1.0,1618387603,Kadaikutty Singam (2018),Action|Children|Comedy|Romance
25000199,162554,152081,5.0,1618387604,Zootopia (2016),Action|Adventure|Animation|Children|Comedy
25000200,162554,8961,5.0,1618387605,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy
25000201,162554,194218,1.0,1618387606,Wickie the Viking (2009),Action|Adventure|Children|Comedy
25000202,162554,26340,3.0,1618387607,"Twelve Tasks of Asterix, The (Les douze travau...",Action|Adventure|Animation|Children|Comedy|Fan...
25000203,162554,1261,2.0,1618387608,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror
25000204,162554,189203,3.0,1618387609,Upgrade (2018),Action|Comedy|Horror|Sci-Fi|Thriller


In [ ]:
print('#Total dataset: %d' % len(movie_dataframe))

movie_train_df, movie_test_df = sk_train_test_split(movie_dataframe,
                                   stratify=movie_dataframe['userId'],
                                   test_size=0.2,
                                   random_state=42)
print('#Train set: %d' % len(movie_train_df))
print('#Test set: %d' % len(movie_test_df))

#Total dataset: 25000225
#Train set: 20000180
#Test set: 5000045


In [ ]:
class ColaborativeFilteringRecommender:
    def __init__(self, train_data):
        self.train_data = train_data
        self.algo = None

    def prepare_data(self):
        # Definir o esquema dos dados de entrada
        reader = Reader(rating_scale=(self.train_data['rating'].min(), self.train_data['rating'].max()))
        data = Dataset.load_from_df(self.train_data[['userId', 'movieId', 'rating']], reader)
        return data

    def train(self):
        data = self.prepare_data()
        trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

        # Treinar o algoritmo SVD
        self.algo = SVD()
        self.algo.fit(trainset)

        # Avaliar o modelo
        self.evaluate(testset)

        joblib.dump(self.algo, 'cf_model.joblib')

    def evaluate(self, testset):
        # Avaliar o modelo no conjunto de teste
        predictions = self.algo.test(testset)

        # RMSE e MAE
        rmse = accuracy.rmse(predictions)
        mae = accuracy.mae(predictions)

        # Precision e Recall
        precisions, recalls = self.precision_recall_at_k(predictions, k=5, threshold=3.5)

        # Média de precisão e recall
        mean_precision = sum(prec for prec in precisions.values()) / len(precisions)
        mean_recall = sum(rec for rec in recalls.values()) / len(recalls)

        print(f'EQMR: {rmse}')
        print(f'EMA: {mae}')
        print(f'Precisão: {mean_precision}')
        print(f'Revocação: {mean_recall}')

    def precision_recall_at_k(self, predictions, k=10, threshold=3.5):
        # Função para calcular precisão e recall em k
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))

        precisions = {}
        recalls = {}

        for uid, user_ratings in user_est_true.items():
            # Ordenar as previsões do usuário
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            # Considerar as top-k previsões
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        return precisions, recalls

    def predict(self, userId, movieId):
        # Prever a avaliação de um usuário para um filme específico
        return self.algo.predict(userId, movieId).est

    def recommend(self, userId, n=10):
        # Obter recomendações para um usuário
        user_movies = self.train_data[self.train_data['userId'] == userId]['movieId'].tolist()
        all_movies = self.train_data['movieId'].unique()
        predictions = [(movieId, self.predict(userId, movieId)) for movieId in all_movies if movieId not in user_movies]
        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:n]

In [ ]:
cf_recommender = ColaborativeFilteringRecommender(movie_dataframe)
cf_recommender.train()
recommendations = cf_recommender.recommend(userId=1, n=10)
print(recommendations)

RMSE: 0.7777
MAE:  0.5869
EQMR: 0.777675454662334
EMA: 0.5869260887527686
Precisão: 0.8231424852547558
Revocação: 0.42404595409565854
[(159817, 4.49257440314969), (2575, 4.48876218239284), (1247, 4.484543007973137), (36363, 4.464871999307328), (2997, 4.4608890851576986), (89759, 4.446312301805543), (2858, 4.440347960849769), (2959, 4.437267235468439), (127180, 4.430327156045361), (171495, 4.42535291311857)]


In [ ]:
class CNNRecommender:
    def __init__(self, cf_model):
        self.cf_model = cf_model
        self.cnn_model = None
        self.user_map = {}
        self.movie_map = {}

    def prepare_data(self):
        # Prepare os dados usando o modelo de filtro colaborativo
        data = self.cf_model.prepare_data()
        trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

        user_ids = self.cf_model.train_data['userId'].unique()
        movie_ids = self.cf_model.train_data['movieId'].unique()

        n_users = len(user_ids)
        n_movies = len(movie_ids)

        self.user_map = {user_id: idx for idx, user_id in enumerate(user_ids)}
        self.movie_map = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

        X_train = []
        y_train = []

        for _, row in self.cf_model.train_data.iterrows():
            user_idx = self.user_map[row['userId']]
            movie_idx = self.movie_map[row['movieId']]
            rating = row['rating']
            X_train.append([user_idx, movie_idx])
            y_train.append(rating)

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        X_test = []
        y_test = []

        for (uid, mid, true_r) in testset:
            if uid in self.user_map and mid in self.movie_map:
                user_idx = self.user_map[uid]
                movie_idx = self.movie_map[mid]
                X_test.append([user_idx, movie_idx])
                y_test.append(true_r)

        X_test = np.array(X_test)
        y_test = np.array(y_test)

        return X_train, y_train, X_test, y_test

    def train(self):
        X_train, y_train, X_test, y_test = self.prepare_data()

        # Construir o modelo CNN
        n_users = X_train[:, 0].max() + 1
        n_movies = X_train[:, 1].max() + 1

        self.cnn_model = Sequential()
        self.cnn_model.add(Embedding(input_dim=n_users, output_dim=50, input_length=2))
        self.cnn_model.add(ZeroPadding1D(padding=1))
        self.cnn_model.add(Conv1D(32, 3, activation='relu'))
        self.cnn_model.add(MaxPooling1D(2))
        self.cnn_model.add(Flatten())
        self.cnn_model.add(Dense(64, activation='relu'))
        self.cnn_model.add(Dense(1))

        self.cnn_model.compile(optimizer='adam', loss='mean_squared_error')

        self.cnn_model.fit(X_train, y_train, epochs=2, batch_size=32)

        self.cnn_model.save("cnn_model.h5")
        joblib.dump(self.user_map, 'user_map.pkl')
        joblib.dump(self.movie_map, 'movie_map.pkl')

        self.evaluate(X_test, y_test)

    def evaluate(self, X_test, y_test):
        # Fazendo previsões
        y_pred = self.cnn_model.predict(X_test)

        # Calcular RMSE e MAE
        rmse = np.sqrt(np.mean((y_test - y_pred.flatten()) ** 2))
        mae = np.mean(np.abs(y_test - y_pred.flatten()))

        print(f'EQMR: {rmse}')
        print(f'EMA: {mae}')

        # Calculando Precision e Recall
        y_pred_binary = [1 if pred >= 3.5 else 0 for pred in y_pred.flatten()]
        y_true_binary = [1 if true >= 3.5 else 0 for true in y_test]

        precision = sum([1 for yt, yp in zip(y_true_binary, y_pred_binary) if yt == 1 and yp == 1]) / sum(y_pred_binary)
        recall = sum([1 for yt, yp in zip(y_true_binary, y_pred_binary) if yt == 1 and yp == 1]) / sum(y_true_binary)

        print(f'Precisão: {precision}')
        print(f'Revocação: {recall}')

    def predict(self, userId, movieId):
        user_idx = self.user_map[userId]
        movie_idx = self.movie_map[movieId]
        return self.cnn_model.predict([[user_idx, movie_idx]]).flatten()[0]

    def recommend(self, userId, n=10):
      user_idx = self.user_map[userId]
      all_movies = self.cf_model.train_data['movieId'].unique()
      watched_movies = self.cf_model.train_data[
          self.cf_model.train_data['userId'] == userId
      ]['movieId'].tolist()

      # Filtrar filmes que o usuário não assistiu
      candidates = [movie_id for movie_id in all_movies if movie_id not in watched_movies]

      # Mapear IDs para índices
      candidate_indices = [
          [user_idx, self.movie_map[movie_id]] for movie_id in candidates if movie_id in self.movie_map
      ]

      if not candidate_indices:
          return []

      # Converter para numpy array
      candidate_indices = np.array(candidate_indices)

      # Fazer predições de uma só vez
      predictions = self.cnn_model.predict(candidate_indices, batch_size=32, verbose=0).flatten()

      # Combinar filmes com suas predições
      movie_predictions = list(zip(candidates, predictions))

      # Ordenar por predições e retornar os top-n
      movie_predictions.sort(key=lambda x: x[1], reverse=True)
      top_n_predictions = movie_predictions[:n]

      # Obter títulos dos filmes
      movie_titles = self.cf_model.train_data[['movieId', 'title']].drop_duplicates()
      id_to_title = dict(zip(movie_titles['movieId'], movie_titles['title']))

      # Retornar tuplas (título, ID, predição)
      recommendations = [(id_to_title[movie_id], movie_id, score) for movie_id, score in top_n_predictions]
      return recommendations

    def calculate_score(self, genres, preferred_genres):
      # Calcula o match entre os generos do novo usuário e os generos existentes
      movie_genres = set(genres.split('|'))
      common_genres = movie_genres.intersection(preferred_genres)
      return len(common_genres)

    def recommend_movies_for_new_user(self, preferred_genres, n=10):
        # Acessar dados do modelo colaborativo
        data = self.cf_model.train_data

        # Adicionar coluna de pontuação com base nos gêneros
        data['score'] = data['genres'].apply(lambda x: self.calculate_score(x, set(preferred_genres)))

        # Filtrar filmes pelos generos que o novo usuário gosta
        filtered_movies = data[data['score'] > 0]

        # Agrupar filmes e gerar média de avaliações
        average_ratings = (
            filtered_movies.groupby(['movieId', 'title', 'score'])['rating']
            .mean()
            .reset_index()
        )

        sorted_movies = average_ratings.sort_values(by=['score', 'rating'], ascending=[False, False])

        # Obter recomendações para um novo usuário
        top_n_movies = sorted_movies.head(n)

        # Retornar tuplas (título, ID, média de avaliação)
        recommendations = [(row['title'], row['movieId'], row['rating']) for _, row in top_n_movies.iterrows()]
        return recommendations

In [ ]:
cnn_recommender = CNNRecommender(cf_recommender)
cnn_recommender.train()
recommendations = cnn_recommender.recommend(userId=1, n=10)
print(recommendations)
preferred_genres = ["Adventure", "Fantasy", "Drama"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)

Epoch 1/2
781258/781258 [==============================] - 30268s 39ms/step - loss: 0.7362
Epoch 2/2
781258/781258 [==============================] - 28110s 36ms/step - loss: 0.6780


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


156252/156252 [==============================] - 193s 1ms/step
EQMR: 0.8028059243872688
EMA: 0.5994469822836684
Precisão: 0.8219960565119335
Revocação: 0.7838866930929272
[('Shawshank Redemption, The (1994)', 318, 4.4545197), ('Twelve Angry Men (1954)', 175981, 4.4343815), ('I, a Man (1967)', 124416, 4.4209323), ('Lord of the Rings: The Return of the King, The (2003)', 7153, 4.3693023), ('7 Plus Seven (1970)', 50742, 4.3653994), ('Lord of the Rings: The Fellowship of the Ring, The (2001)', 4993, 4.336666), ('Black Mirror', 176601, 4.3211346), ('Open Hearts (Elsker dig for evigt) (2002)', 6192, 4.314093), ('Usual Suspects, The (1995)', 50, 4.306941), ('Planet Earth (2006)', 159817, 4.305239)]
[('Princess Mononoke (Mononoke-hime) (1997)', 3000, 4.138159397122631), ('Lord of the Rings: The Return of the King, The (2003)', 7153, 4.0903399807075225), ('Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)', 7099, 4.087514145605432), ("Kiki's Delivery Service (Majo no takkyûbi

In [ ]:
files.download("cnn_model.h5")
files.download('user_map.pkl')
files.download('movie_map.pkl')

# Salvar o modelo CNN
cnn_recommender.cnn_model.save("cnn_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
movie_dataframe['genres_list'] = movie_dataframe['genres'].str.split('|')
genres_exploded = movie_dataframe.explode('genres_list')
unique_genres = genres_exploded['genres_list'].unique()
unique_genres_list = list(unique_genres)
print(unique_genres_list)

['Comedy', 'Crime', 'Drama', 'Thriller', 'War', 'Musical', 'Romance', 'Adventure', 'Film-Noir', 'Sci-Fi', 'Western', 'Fantasy', 'Mystery', 'Children', 'Action', 'Documentary', 'Animation', 'Horror', 'IMAX', '(no genres listed)']


Isaac Burton

In [ ]:
preferred_genres = ["Adventure", "Fantasy", "Sci-Fi", "Animation", "Horror"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162543, n=5)
print(final_recommendations)

[('Neo Tokyo (1987)', 159403, 3.671875), ('Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)', 7099, 4.087514145605432), ('Laputa: Castle in the Sky (Tenkû no shiro Rapyuta) (1986)', 6350, 4.067602274811961), ('Patema Inverted (2013)', 116905, 3.813888888888889), ('Mind Game (2004)', 47404, 3.811023622047244), ('Interstella 5555: The 5tory of the 5ecret 5tar 5ystem (2003)', 27800, 3.7592105263157896), ('Berserk: The Golden Age Arc - The Egg of the King (2012)', 130508, 3.7263513513513513), ('Vampire Hunter D: Bloodlust (Banpaia hantâ D) (2000)', 5146, 3.681761006289308), ('Galaxy Express 999 (Ginga tetsudô Three-Nine) (1979)', 65796, 3.55), ('Gwen, the Book of Sand (1985)', 140533, 3.5)]
[('Shawshank Redemption, The (1994)', 318, 4.4792433), ('Paperman (2012)', 98491, 4.316662), ('Planet Earth II (2016)', 171011, 4.2968683), ('Intouchables (2011)', 92259, 4.2788715), ('Black Mirror', 176601, 4.2769766)]


Lais Marques

In [ ]:
preferred_genres = ["Crime", "Thriller", "Sci-Fi", "Animation", "Horror"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162542, n=5)
print(final_recommendations)

[('Ghost in the Shell: Solid State Society (2006)', 151781, 3.7409090909090907), ('Grindhouse (2007)', 52281, 3.676338028169014), ('Eden of the East the Movie II: Paradise Lost (Higashi no Eden Gekijôban II: Paradise Lost) (2010)', 93750, 3.452830188679245), ('Cyber City Oedo 808 (1990)', 141367, 3.4375), ('Eden of the East the Movie I: The King of Eden (Higashi no Eden Gekijoban) (2009)', 93748, 3.4285714285714284), ('Hellboy Animated: Blood and Iron (2007)', 140511, 3.3555555555555556), ('King of Thorn (King of Thorns) (Ibara no O) (2009)', 95773, 3.1923076923076925), ('God told Me To (1976)', 6741, 2.941860465116279), ('Deathrow Gameshow (1987)', 194750, 1.0), ('Deadly Signal (2016)', 206785, 0.75)]
[('Planet Earth II (2016)', 171011, 3.8830812), ('Over the Garden Wall (2013)', 163809, 3.833285), ('Shawshank Redemption, The (1994)', 318, 3.814979), ('Black Mirror', 176601, 3.8109293), ('Lives of Others, The (Das leben der Anderen) (2006)', 44555, 3.8051012)]


Thiago Veloso

In [ ]:
preferred_genres = ["Crime", "Thriller", "Comedy", "Animation", "Action"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162551, n=5)
print(final_recommendations)

[('Osmosis Jones (2001)', 4719, 2.873650107991361), ('Monday (2000)', 27328, 3.9375), ('Diamond Arm, The (Brilliantovaya ruka) (1968)', 26184, 3.9318181818181817), ('Lupin III: The Castle Of Cagliostro (Rupan sansei: Kariosutoro no shiro) (1979)', 8253, 3.876916524701874), ('Midnight Run (1988)', 3104, 3.8079810351639667), ('Perfect Murder, The (1988)', 96775, 3.8), ('Ghost in the Shell: Solid State Society (2006)', 151781, 3.7409090909090907), ('Sherlock Holmes: A Game of Shadows (2011)', 91542, 3.7256908904810646), ('Police Story (Ging chaat goo si) (1985)', 26547, 3.6940451745379876), ('Airbag (1997)', 71141, 3.6875)]
[('Shawshank Redemption, The (1994)', 318, 4.380701), ('Black Mirror', 176601, 4.24649), ("Schindler's List (1993)", 527, 4.2340064), ('Paperman (2012)', 98491, 4.2327557), ('Life Is Beautiful (La Vita è bella) (1997)', 2324, 4.2267604)]


Amanda Goudinho

In [ ]:
preferred_genres = ["Sci-Fi", "Animation", "Adventure"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162544, n=5)
print(final_recommendations)

[('Spider-Man: Into the Spider-Verse (2018)', 195159, 4.111633182112767), ('Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)', 7099, 4.087514145605432), ('Laputa: Castle in the Sky (Tenkû no shiro Rapyuta) (1986)', 6350, 4.067602274811961), ('Grand Day Out with Wallace and Gromit, A (1989)', 1223, 4.048011954261955), ('WALL·E (2008)', 60069, 4.0049680365296805), ('Akira (1988)', 1274, 3.9066041848299915), ('Iron Giant, The (1999)', 2761, 3.866325081168831), ('Patema Inverted (2013)', 116905, 3.813888888888889), ('Mind Game (2004)', 47404, 3.811023622047244), ('Summer Wars (Samâ wôzu) (2009)', 85179, 3.7915451895043732)]
[('Shawshank Redemption, The (1994)', 318, 4.1032343), ('Planet Earth II (2016)', 171011, 4.0519276), ('Paperman (2012)', 98491, 3.9983003), ('Planet Earth (2006)', 159817, 3.9905782), ('Band of Brothers (2001)', 170705, 3.9873736)]


Francine Paz

In [ ]:
preferred_genres = ["Drama", "Film-Noir", "Fantasy"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162550, n=5)
print(final_recommendations)

[('Lost Highway (1997)', 1464, 3.722954005614338), ('Tulse Luper Suitcases, Part 1: The Moab Story, The (2003)', 62304, 3.361111111111111), ('The Old Fairy Tale: When the Sun Was God (2003)', 151569, 5.0), ('The Flower (2019)', 200286, 5.0), ('The Lurking Man (2018)', 204488, 5.0), ('Curse of the Oily Man (Sumpah orang minyak) (1956) ', 106460, 4.5), ('The Ice Princess (1996)', 165401, 4.5), ('Hamlet, Prince of Denmark (1961)', 173127, 4.5), ('Edge of the Garden (2011)', 193129, 4.5), ('Arabian Nights (2000)', 176171, 4.333333333333333)]
[('Shawshank Redemption, The (1994)', 318, 4.354934), ('Black Mirror', 176601, 4.3459506), ('Over the Garden Wall (2013)', 163809, 4.2731113), ('Planet Earth (2006)', 159817, 4.2326226), ("Schindler's List (1993)", 527, 4.221948)]


Marcos

In [ ]:
preferred_genres = ["Thriller", "Musical", "Fantasy", "Animation", "Action"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162552, n=5)
print(final_recommendations)

[('Hellboy Animated: Blood and Iron (2007)', 140511, 3.3555555555555556), ('Hellboy Animated: Sword of Storms (2006)', 136451, 2.923076923076923), ('Fate/Stay Night: Unlimited Blade Works (2010)', 196965, 4.25), ('Heavenly Sword (2014)', 199358, 4.166666666666667), ('Princess Mononoke (Mononoke-hime) (1997)', 3000, 4.138159397122631), ('Laputa: Castle in the Sky (Tenkû no shiro Rapyuta) (1986)', 6350, 4.067602274811961), ('I Am So Proud of You (2008)', 174703, 3.9468085106382977), ('Monday (2000)', 27328, 3.9375), ('Berserk: The Golden Age Arc 3 - Descent (2013)', 130510, 3.925), ('Neon Genesis Evangelion: The End of Evangelion (Shin seiki Evangelion Gekijô-ban: Air/Magokoro wo, kimi ni) (1997)', 27156, 3.912202380952381)]
[('Shawshank Redemption, The (1994)', 318, 4.573092), ('Planet Earth II (2016)', 171011, 4.5000496), ('Black Mirror', 176601, 4.4594526), ('Planet Earth (2006)', 159817, 4.450174), ('Over the Garden Wall (2013)', 163809, 4.4104304)]


Thamyres

In [ ]:
preferred_genres = ["Drama", "Comedy", "Romance", "Sci-Fi", "Animation"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)

[('Girl Who Leapt Through Time, The (Toki o kakeru shôjo) (2006)', 57504, 3.935126582278481), ('Princes and Princesses (2000)', 146305, 3.875), ("It's Such a Beautiful Day (2012)", 99764, 4.003694581280788), ('Feast (2014)', 138702, 3.9255852842809364), ('Mind Game (2004)', 47404, 3.811023622047244), ('Revolutionary Girl Utena: Adolescence of Utena (a.k.a. Revolutionary Girl Utena the Movie) (Shoujo kakumei Utena: Adolescence mokushiroku) (1999)', 27344, 3.742857142857143), ('Mulan (1998)', 1907, 3.731475645989016), ('The Web (2013)', 192233, 3.6666666666666665), ('Sailor Moon R: The Movie (1993)', 140809, 3.578125), ("I'm a Cyborg, But That's OK (Saibogujiman kwenchana) (2006)", 53161, 3.5742924528301887)]


Margarida

In [ ]:
preferred_genres = ["Drama", "Thriller", "Fantasy", "Sci-Fi", "Horror"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162553, n=5)
print(final_recommendations)

[('Gamera 2: Attack of the Legion (1996)', 194979, 3.4166666666666665), ('Faust: Love of the Damned (2001)', 187043, 2.85), ('Q: The Winged Serpent (1982)', 6120, 2.8106060606060606), ('Pulse (2006)', 43932, 2.4501557632398754), ("Ba'al (2008)", 167282, 2.3333333333333335), ('Black Night (2006)', 188983, 4.5), ('Interstate 60 (2002)', 6902, 3.9292828685258963), ("Encounter at Raven's Gate (1988)", 170397, 3.75), ("Devil's Backbone, The (Espinazo del diablo, El) (2001)", 4902, 3.7323901264298613), ('Videodrome (1983)', 6774, 3.6975206611570246)]
[('Black Mirror', 176601, 4.2816987), ('Shawshank Redemption, The (1994)', 318, 4.266924), ('Over the Garden Wall (2013)', 163809, 4.2547264), ('Lives of Others, The (Das leben der Anderen) (2006)', 44555, 4.1808486), ('Planet Earth (2006)', 159817, 4.1798654)]


Júlia Ruas

In [ ]:
preferred_genres = ["Children", "Comedy", "Action", "Documentary", "Horror"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162554, n=5)
print(final_recommendations)

[('American Scary (2006)', 108316, 4.0), ('Kings of Kallstadt (2014)', 169858, 4.0), ('Zootopia (2016)', 152081, 3.8937737369087353), ('Incredibles, The (2004)', 8961, 3.854922618852861), ("Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976)", 26340, 3.802788844621514), ('Evil Dead II (Dead by Dawn) (1987)', 1261, 3.7880156211862337), ('Upgrade (2018)', 189203, 3.763809523809524), ('Zombieland (2009)', 71535, 3.7629512767830935), ('Army of Darkness (1993)', 1215, 3.7330416633218273), ("Pusher III: I'm the Angel of Death (2005)", 51545, 3.6891025641025643)]
[('Shawshank Redemption, The (1994)', 318, 4.3133526), ('Black Mirror', 176601, 4.201962), ('Planet Earth II (2016)', 171011, 4.1995134), ('Band of Brothers (2001)', 170705, 4.176365), ('Planet Earth (2006)', 159817, 4.167618)]


Yasmin

In [ ]:
preferred_genres = ["Comedy", "Thriller", "Romance", "Horror"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162545, n=5)
print(final_recommendations)

[('Pizza (2012)', 135585, 3.625), ('Mourning Grave (2014)', 144494, 3.392857142857143), ('Pride and Prejudice and Zombies (2016)', 149830, 2.897689768976898), ('Svidd Neger (2003)', 27781, 2.85), ('Nature of the Beast (2007)', 172281, 2.75), ('Love Object (2003)', 27658, 2.7083333333333335), ('American Werewolf in Paris, An (1997)', 2793, 2.643202709240445), ('Modern Vampires (1998)', 136946, 1.75), ('What a Carve Up! (1961)', 134306, 4.25), ('Blood Moon (2016)', 161776, 4.25)]
[('Shawshank Redemption, The (1994)', 318, 4.361931), ('Black Mirror', 176601, 4.3156004), ('Over the Garden Wall (2013)', 163809, 4.2339187), ('Lives of Others, The (Das leben der Anderen) (2006)', 44555, 4.2070894), ("Schindler's List (1993)", 527, 4.2006845)]


Sue Marques

In [ ]:
preferred_genres = ["Thriller", "Documentary", "Drama", "Children", "Animation"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162549, n=5)
print(final_recommendations)

[('Old Partner (2008)', 168918, 4.3), ('My Neighbor Totoro (Tonari no Totoro) (1988)', 5971, 4.152446204903115), ("Kiki's Delivery Service (Majo no takkyûbin) (1989)", 26662, 4.010751496342275), ('Up (2009)', 68954, 3.96362623368354), ('Inside Out (2015)', 134853, 3.932331934320006), ('Feast (2014)', 138702, 3.9255852842809364), ('Waltz with Bashir (Vals im Bashir) (2008)', 61236, 3.8876511775938893), ('Princes and Princesses (2000)', 146305, 3.875), ('Iron Giant, The (1999)', 2761, 3.866325081168831), ('Drishyam (2013)', 120311, 3.8421052631578947)]
[('Shawshank Redemption, The (1994)', 318, 4.4073167), ('Black Mirror', 176601, 4.347472), ('Over the Garden Wall (2013)', 163809, 4.2660728), ("Schindler's List (1993)", 527, 4.2579975), ('Planet Earth (2006)', 159817, 4.2473574)]


Beatriz

In [ ]:
preferred_genres = ["Romance", "Sci-Fi", "Fantasy", "Action", "Animation"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)

[('Laputa: Castle in the Sky (Tenkû no shiro Rapyuta) (1986)', 6350, 4.067602274811961), ('Neon Genesis Evangelion: The End of Evangelion (Shin seiki Evangelion Gekijô-ban: Air/Magokoro wo, kimi ni) (1997)', 27156, 3.912202380952381), ('Princes and Princesses (2000)', 146305, 3.875), ('Mind Game (2004)', 47404, 3.811023622047244), ('Revolutionary Girl Utena: Adolescence of Utena (a.k.a. Revolutionary Girl Utena the Movie) (Shoujo kakumei Utena: Adolescence mokushiroku) (1999)', 27344, 3.742857142857143), ('Vampires in Havana (1985)', 175997, 3.5), ('Doctor Strange (2007)', 115927, 3.4939759036144578), ('Green Snake (1993)', 141133, 3.4615384615384617), ('Appleseed (Appurushîdo) (2004)', 31184, 3.4522332506203472), ('Sword Art Online The Movie: Ordinal Scale (2017)', 177285, 3.4166666666666665)]


Neylon Campelo

In [ ]:
preferred_genres = ["Thriller", "Drama", "Crime", "Fantasy", "Adventure"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162547, n=5)
print(final_recommendations)

[('The Stone Council (2006)', 128985, 3.5), ('Motorama (1991)', 5018, 3.0444444444444443), ('City of God (Cidade de Deus) (2002)', 6016, 4.181628549886907), ('Killer by Nature (2010)', 175047, 4.0), ('Interstate 60 (2002)', 6902, 3.9292828685258963), ('Blood Diamond (2006)', 49530, 3.8444281285690978), ('Defiant Ones, The (1958)', 4998, 3.7784615384615385), ("Honor Among Thieves (Adieu l'ami) (Farewell, Friend) (1968)", 59844, 3.727272727272727), ('Dragnet (1954)', 183347, 3.6666666666666665), ('Prairie Dog (2015)', 201869, 3.6666666666666665)]
[('Over the Garden Wall (2013)', 163809, 4.3243904), ('Shawshank Redemption, The (1994)', 318, 4.312808), ('Godfather, The (1972)', 858, 4.299437), ('Planet Earth II (2016)', 171011, 4.2958827), ('Black Mirror', 176601, 4.295102)]


Maurício Ruas

In [ ]:
preferred_genres = ["Comedy", "War", "Sci-Fi"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162546, n=5)
print(final_recommendations)

[('Slaughterhouse-Five (1972)', 8690, 3.673559822747415), ("Tomorrow I'll Wake Up and Scald Myself with Tea (1977)", 173707, 3.357142857142857), ('Embassy (2013)', 171811, 3.25), ('Glamorous Life of Sachiko Hanai, The (Hatsujô kateikyôshi: sensei no aijiru) (2003)', 97724, 3.066666666666667), ('Jackboots on Whitehall (2011)', 116819, 2.6), ('Invisible Agent (1942)', 79681, 2.5), ('From Other Worlds (2004)', 197095, 5.0), ("You're in the Navy Now (1951)", 165987, 4.5), ('The Cow and I (1959)', 188795, 4.333333333333333), ('Bewitched (1981)', 197005, 4.25)]
[('Lives of Others, The (Das leben der Anderen) (2006)', 44555, 4.265527), ('Planet Earth (2006)', 159817, 4.263535), ('Over the Garden Wall (2013)', 163809, 4.253799), ('Planet Earth II (2016)', 171011, 4.217828), ('Shawshank Redemption, The (1994)', 318, 4.2030573)]


Jeniffer

In [ ]:
preferred_genres = ["Comedy", "War", "Romance", "Documentary", "Action"]
recommendations_new_user = cnn_recommender.recommend_movies_for_new_user(preferred_genres, n=10)
print(recommendations_new_user)
final_recommendations = cnn_recommender.recommend(userId=162548, n=5)
print(final_recommendations)

[('White Sun of the Desert, The (Beloe solntse pustyni) (1970)', 26236, 3.7842465753424657), ('Operation Petticoat (1959)', 4802, 3.651424287856072), ('Fanfan la Tulipe (Fan-Fan the Tulip) (1952)', 70223, 3.625), ('Hot Shots! (1991)', 5541, 3.265979689366786), ('Big Noise, The (1944)', 115862, 3.1666666666666665), ('Ghosts of Cité Soleil (2006)', 68831, 3.066666666666667), ('Berlin Tunnel 21 (1981)', 137176, 2.75), ('Lazybones (1925)', 87745, 2.6666666666666665), ('Mr. & Mrs. Incredible (2011)', 141731, 4.5), ('Thiruda Thirudi (2003)', 208183, 4.5)]
[('Lives of Others, The (Das leben der Anderen) (2006)', 44555, 4.1146812), ('Over the Garden Wall (2013)', 163809, 4.0788264), ('Black Mirror', 176601, 4.056471), ('Planet Earth II (2016)', 171011, 4.0455437), ('Shawshank Redemption, The (1994)', 318, 4.041651)]
